In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf
import ptb_reader
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.models import load_model

import os
from __future__ import division

Using TensorFlow backend.


In [3]:
batch_size = 1000
num_steps = 20
hidden_size = 70
feat_len = 49

In [4]:
def get_model():
    
    model = Sequential()
    # , dropout=0.2, recurrent_dropout=0.2
    model.add(LSTM(hidden_size, return_sequences=True, input_shape=( num_steps, feat_len)))
    model.add(LSTM(hidden_size))
    model.add(Dense(vocab, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

Build model...


In [5]:
# model.add(Embedding(max_features, num_steps))
# model.add(LSTM(50, input_shape=(1,1), dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(vocab, activation='softmax'))
# model.pop()
# model.add(Dense(10, input_shape=(10,), activation='softmax'))

# model.pop()
# model.pop()



In [6]:


# plot_model(model, to_file='model.png')

In [7]:
def train(verbose = 2):

    model = get_model()
    
    if(verbose > 0):
        print('Train...')

    for step, (x, y) in enumerate(ptb_reader.ptb_iterator(train_data, batch_size, num_steps)):    
        x1 = np.zeros((batch_size, num_steps, feat_len))
        for i in range(x1.shape[0]):
            for j in range(x1.shape[1]):
                x1[i,j,y[i,-1]] = 1

        y1 = y[:,-1]

        model.fit(x1, y1,epochs=1, verbose = verbose)

        if(step % 100 == 0 and verbose > 2):
            print(step+1, end=' ')
        
        
    if( not os.path.isdir('weights') ):
        os.mkdir('weights')
        
    model.save('weights/my_model.h5')
    
#     model.save_weights('my_model_weights.h5')
    
    

Train...
Epoch 1/1
3s - loss: 3.1518 - acc: 0.4210
1 Epoch 1/1
2s - loss: 1.1818 - acc: 0.7470
Epoch 1/1
2s - loss: 0.2982 - acc: 0.9250
Epoch 1/1
2s - loss: 0.1175 - acc: 0.9830
Epoch 1/1
2s - loss: 0.0906 - acc: 0.9890
Epoch 1/1
2s - loss: 0.0550 - acc: 0.9920
Epoch 1/1
2s - loss: 0.0755 - acc: 0.9830
Epoch 1/1
2s - loss: 0.0318 - acc: 0.9940
Epoch 1/1
3s - loss: 0.0277 - acc: 0.9980
Epoch 1/1
3s - loss: 0.0215 - acc: 0.9980
Epoch 1/1
3s - loss: 0.0246 - acc: 0.9990
Epoch 1/1
3s - loss: 0.0205 - acc: 0.9970
Epoch 1/1
3s - loss: 0.0144 - acc: 0.9970
Epoch 1/1
3s - loss: 0.0130 - acc: 0.9980
Epoch 1/1
3s - loss: 0.0219 - acc: 0.9970
Epoch 1/1
4s - loss: 0.0069 - acc: 1.0000
Epoch 1/1
3s - loss: 0.0144 - acc: 0.9990
Epoch 1/1
3s - loss: 0.0244 - acc: 0.9980
Epoch 1/1
3s - loss: 0.0084 - acc: 1.0000
Epoch 1/1
3s - loss: 0.0071 - acc: 1.0000
Epoch 1/1
3s - loss: 0.0078 - acc: 0.9990
Epoch 1/1
3s - loss: 0.0166 - acc: 0.9970
Epoch 1/1
3s - loss: 0.0081 - acc: 1.0000
Epoch 1/1
3s - loss: 0.

True


In [13]:
def test():

    model =load_model('weights/my_model.h5')

#     model = get_model()
#     model.load_weights('my_model_weights.h5')
    
    accuracy = 0.0
    siz = 0
    for step, (x, y) in enumerate(ptb_reader.ptb_iterator(test_data, batch_size, num_steps)):
        x1 = np.zeros((batch_size, num_steps, feat_len))

        for i in range(x1.shape[0]):
            for j in range(x1.shape[1]):
                x1[i,j,y[i,-1]] = 1

        output = model.predict(x1)
        y_hat = np.argmax(output, axis=1)
        acc = np.mean( np.equal(y[:,-1], y_hat))
        accuracy += acc
        siz += 1 
    
        print('#Step: ',step+1, end='')
        print(', Test accuracy:', acc)

    print('Averaged accuracy: ', accuracy/siz)

#Step:  1, Test accuracy: 1.0
#Step:  2, Test accuracy: 1.0
#Step:  3, Test accuracy: 0.999
#Step:  4, Test accuracy: 0.999
#Step:  5, Test accuracy: 1.0
#Step:  6, Test accuracy: 1.0
#Step:  7, Test accuracy: 0.999
#Step:  8, Test accuracy: 1.0
#Step:  9, Test accuracy: 1.0
#Step:  10, Test accuracy: 1.0
#Step:  11, Test accuracy: 1.0
#Step:  12, Test accuracy: 1.0
#Step:  13, Test accuracy: 1.0
#Step:  14, Test accuracy: 1.0
#Step:  15, Test accuracy: 1.0
#Step:  16, Test accuracy: 1.0
#Step:  17, Test accuracy: 0.999
#Step:  18, Test accuracy: 1.0
#Step:  19, Test accuracy: 0.999
#Step:  20, Test accuracy: 1.0
#Step:  21, Test accuracy: 1.0
#Step:  22, Test accuracy: 1.0
#Step:  23, Test accuracy: 1.0
Averaged accuracy:  0.999782608696


In [ ]:
if __name__ == "__main__":
    global feat_len
    
    train_data, valid_data, test_data, vocab = ptb_reader.ptb_raw_data("../data")
    feat_len = vocab
    
    train()
    test()